In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [17]:
# urls are sorted as 'Queen bed', '3 stars', and 'Hotels' for each city
# date to be modified after 
checkin = "2024-12-23"
checkout = "2024-12-24"

min_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Minneapolis%2C+Minnesota%2C+United+States&ssne=Boston&ssne_untouched=Boston&lang=en-us&src=searchresults&dest_id=20067845&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=8c9d9d647dde0344&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
chi_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Chicago%2C+Illinois%2C+United+States&ssne=Minneapolis&ssne_untouched=Minneapolis&lang=en-us&src=searchresults&dest_id=20033173&dest_type=city&ac_position=1&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=b32a9dbd5dc602f9&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
det_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=detroit&ssne=Minneapolis&ssne_untouched=Minneapolis&lang=en-us&src=searchresults&dest_id=20064402&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=54b49e47d6aa01b5&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
gb_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Green+Bay%2C+Wisconsin%2C+United+States&ssne=Detroit&ssne_untouched=Detroit&lang=en-us&src=searchresults&dest_id=20150048&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=229b9e5c71980024&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
nfc_north = [min_url, chi_url, det_url, gb_url]
nfc_north_codes = ['MIN', 'CHI', 'DET', 'GB']

phi_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Philadelphia%2C+Pennsylvania%2C+United+States&ssne=Green+Bay&ssne_untouched=Green+Bay&efdco=1&lang=en-us&src=searchresults&dest_id=20111994&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=86ed9ed6596c0237&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
wsh_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Washington%2C+District+of+Columbia%2C+United+States&ssne=Landover&ssne_untouched=Landover&lang=en-us&src=searchresults&dest_id=20021296&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=38a4803b567c05c8&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
dal_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Dallas%2C+Texas%2C+United+States&ssne=Washington%2C+D.C.&ssne_untouched=Washington%2C+D.C.&lang=en-us&src=searchresults&dest_id=20127504&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=5dde8046957602dd&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
nyg_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=New+York%2C+New+York%2C+United+States&ssne=East+Rutherford&ssne_untouched=East+Rutherford&lang=en-us&src=searchresults&dest_id=20088325&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=5f36808162fd04bf&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
nfc_east = []
nfc_east_codes = ['PHI', 'WSH', 'DAL', 'NYG']

tb_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Tampa%2C+Florida%2C+United+States&ssne=Tampa+Bay+Harbor&ssne_untouched=Tampa+Bay+Harbor&lang=en-us&src=searchresults&dest_id=20024246&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=b42580b72f400119&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
atl_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Atlanta%2C+Georgia%2C+United+States&ssne=Tampa&ssne_untouched=Tampa&lang=en-us&src=searchresults&dest_id=20024809&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=d59680c8e1b1027f&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
no_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=New+Orleans%2C+Louisiana%2C+United+States&ssne=Atlanta&ssne_untouched=Atlanta&lang=en-us&src=searchresults&dest_id=20050264&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=913580d222fe02d2&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
car_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Charlotte%2C+North+Carolina%2C+United+States&ssne=New+Orleans&ssne_untouched=New+Orleans&lang=en-us&src=searchresults&dest_id=20091627&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=916b80e6a5cd0206&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
nfc_south = []
nfc_south_codes = ['TB', 'ATL', 'NO', 'CAR']

lar_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Los+Angeles%2C+California%2C+United+States&ssne=Charlotte&ssne_untouched=Charlotte&lang=en-us&src=searchresults&dest_id=20014181&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=72b28101847901b5&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
sea_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Seattle%2C+Washington%2C+United+States&ssne=Los+Angeles&ssne_untouched=Los+Angeles&lang=en-us&src=searchresults&dest_id=20144883&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=5f4c811495c100dd&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
ari_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Glendale%2C+Arizona%2C+United+States&ssne=Seattle&ssne_untouched=Seattle&lang=en-us&src=searchresults&dest_id=20006308&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=d596812cd1ca04b7&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
sf_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Santa+Clara%2C+California%2C+United+States&ssne=Glendale&ssne_untouched=Glendale&lang=en-us&src=searchresults&dest_id=20015795&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=190c8148808a01a6&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
nfc_west = []
nfc_west_codes = ['LAR', 'SEA', 'ARI', 'SF']

kc_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Kansas+City&ssne=Kansas+City&ssne_untouched=Kansas+City&lang=en-us&src=searchresults&dest_id=20073662&dest_type=city&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
lac_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Los+Angeles%2C+California%2C+United+States&ssne=Charlotte&ssne_untouched=Charlotte&lang=en-us&src=searchresults&dest_id=20014181&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=72b28101847901b5&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
den_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Denver%2C+Colorado%2C+United+States&ssne=Kansas+City&ssne_untouched=Kansas+City&lang=en-us&src=searchresults&dest_id=20017349&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=a6f18186f42c01e1&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
lv_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Las+Vegas%2C+Nevada%2C+United+States&ssne=Denver&ssne_untouched=Denver&lang=en-us&src=searchresults&dest_id=20079110&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=a8038198f3dc0259&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
afc_west = []
afc_west_codes = ['KC', 'LAC', 'DEN', 'LV']

buf_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Buffalo%2C+New+York%2C+United+States&ssne=Las+Vegas&ssne_untouched=Las+Vegas&lang=en-us&src=searchresults&dest_id=20085250&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=8aa081a52c4702af&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
mia_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Miami+Beach%2C+Florida%2C+United+States&ssne=Buffalo&ssne_untouched=Buffalo&lang=en-us&src=searchresults&dest_id=20023182&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=b39081c59a1700fa&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
nyj_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=New+York%2C+New+York%2C+United+States&ssne=East+Rutherford&ssne_untouched=East+Rutherford&lang=en-us&src=searchresults&dest_id=20088325&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=5f36808162fd04bf&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
ne_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Boston%2C+Massachusetts%2C+United+States&ssne=Foxborough&ssne_untouched=Foxborough&lang=en-us&src=searchresults&dest_id=20061717&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=190c81ec77540407&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
afc_east = []
afc_east_codes = ['BUF', 'MIA', 'NYJ', 'NE']

hou_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Houston%2C+Texas%2C+United+States&ssne=Boston&ssne_untouched=Boston&lang=en-us&src=searchresults&dest_id=20128761&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=67c481ffee4104b3&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
ind_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Indianapolis%2C+Indiana%2C+United+States&ssne=Houston&ssne_untouched=Houston&lang=en-us&src=searchresults&dest_id=20037880&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=dd98820ef4fd0193&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
jax_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Jacksonville%2C+Florida%2C+United+States&ssne=Indianapolis&ssne_untouched=Indianapolis&lang=en-us&src=searchresults&dest_id=20022757&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=b32a821c6e64047c&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
ten_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Nashville&ssne=Nashville&ssne_untouched=Nashville&lang=en-us&src=searchresults&dest_id=20123908&dest_type=city&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
afc_south = []
afc_south_codes = ['HOU', 'IND', 'JAX', 'TEN']

pit_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Pittsburgh%2C+Pennsylvania%2C+United+States&ssne=Nashville&ssne_untouched=Nashville&lang=en-us&src=searchresults&dest_id=20112087&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=cd7f824525240e1a&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
bal_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Baltimore%2C+Maryland%2C+United+States&ssne=Pittsburgh&ssne_untouched=Pittsburgh&lang=en-us&src=searchresults&dest_id=20053799&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=5ac1825149da03c8&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=class%3D3%3Bht_id%3D204%3Btdb%3D5"
cin_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Cincinnati%2C+Ohio%2C+United+States&ssne=Baltimore&ssne_untouched=Baltimore&lang=en-us&src=searchresults&dest_id=20097593&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=4f70825f070c00dd&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
cle_url = f"https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Cleveland%2C+Ohio%2C+United+States&ssne=Cincinnati&ssne_untouched=Cincinnati&lang=en-us&src=searchresults&dest_id=20097630&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=00c5826b1db503e6&checkin={checkin}&checkout={checkout}&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
afc_north = []
afc_north_codes = ['PIT', 'BAL', 'CIN', 'CLE']

all_urls = [min_url, chi_url, det_url, gb_url, phi_url, wsh_url, dal_url, nyg_url, tb_url, no_url, car_url, atl_url,
            lar_url, ari_url, sf_url, sea_url, kc_url, lac_url, den_url, lv_url, buf_url, mia_url, nyj_url, ne_url, 
            hou_url, ind_url, jax_url, ten_url, pit_url, bal_url, cin_url, cle_url]
all_codes = afc_east_codes + afc_north_codes + afc_south_codes + afc_west_codes + nfc_east_codes + nfc_north_codes + nfc_west_codes + nfc_south_codes

['BUF', 'MIA', 'NYJ', 'NE', 'PIT', 'BAL', 'CIN', 'CLE', 'HOU', 'IND', 'JAX', 'TEN', 'KC', 'LAC', 'DEN', 'LV', 'PHI', 'WSH', 'DAL', 'NYG', 'MIN', 'CHI', 'DET', 'GB', 'LAR', 'SEA', 'ARI', 'SF', 'TB', 'ATL', 'NO', 'CAR']


In [8]:
# webscrapes the hotel prices, name, room type, and rating
def hotel_webscraper():
    
    l=list()
    o={}
    headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
    resp = requests.get(i, headers=headers)


200
200
200
200


In [31]:
def scrape_hotel_details(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, "html.parser")

    # List to store hotel details
    hotel_details = []

    # Gets number of properties that are options
    properties_text = soup.select_one(".f6431b446c.d5f78961c3").get_text(strip=True)  # Example class (adjust as needed)
    properties_match = re.search(r"(\d+)\s+properties found", properties_text)
    num_properties = int(properties_match.group(1)) if properties_match else 0

    print(f"Number of properties found: {num_properties}")

    # Select the main container for hotel listings
    hotels = soup.select(".c624d7469d.a0e60936ad.a3214e5942.b0db0e8ada")  # Example class for hotel cards (may vary)

    for hotel in hotels[:num_properties]:
        try:
            # Extract hotel name
            name_element = hotel.select_one(".f6431b446c.a15b38c233")  # Example class for name
            hotel_name = name_element.get_text(strip=True) if name_element else "N/A"

            # Extract price
            price_element = hotel.select_one(".f6431b446c.fbfd7c1165.e84eb96b1f")  # Example class for price
            price_text = price_element.get_text(strip=True) if price_element else "N/A"
            hotel_price = int("".join(filter(str.isdigit, price_text))) if price_text != "N/A" else "N/A"

            # Extract rating
            rating_element = hotel.select_one(".c624d7469d.eb03ae5461.dab7c5c6fa.a937b09340.a3214e5942.d5fd510f01.dc7f26e57f")  # Example class for rating
            hotel_rating = rating_element.get_text(strip=True) if rating_element else "N/A"

            # Extract room type
            room_type_element = hotel.select_one(".abf093bdfe")  # Example class for room type
            room_type = room_type_element.get_text(strip=True) if room_type_element else "N/A"

            # Append the details to the list
            hotel_details.append({
                "Name": hotel_name,
                "Price": hotel_price,
                "Rating": hotel_rating,
                "Room Type": room_type
            })
        except Exception as e:
            print(f"Error processing hotel: {e}")
            continue

    return hotel_details

# Example URL
booking_url = "https://www.booking.com/searchresults.html?label=gen173bo-1DEg1mbGlnaHRzX2luZGV4KIICQgVpbmRleEgxWANoiQKIAQGYATG4AQfIAQzYAQPoAQH4AQaIAgGYAgKoAgO4Arfcl7sGwAIB0gIkYWY4MjUwNWYtOTY0NC00ZWJiLWIwYjQtYjcxM2I2OGI4YjYz2AIE4AIB&sid=064de026aa69e51a79ebeb022d0c60c1&aid=304142&ss=Cleveland%2C+Ohio%2C+United+States&ssne=Cincinnati&ssne_untouched=Cincinnati&lang=en-us&src=searchresults&dest_id=20097630&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=00c5826b1db503e6&checkin=2024-12-27&checkout=2024-12-29&group_adults=2&no_rooms=1&group_children=0&nflt=tdb%3D5%3Bclass%3D3%3Bht_id%3D204"
hotels = scrape_hotel_details(booking_url)

if hotels:
    for hotel in hotels:
        print(hotel)
else:
    print("No hotel details found.")


Number of properties found: 10
{'Name': 'Drury Plaza Hotel Cleveland Downtown', 'Price': 299, 'Rating': 'Scored 9.09.0Wonderful2,188 reviews', 'Room Type': 'Downtown Cleveland, ClevelandShow on map0.2 miles from downtown'}
{'Name': 'Holiday Inn Cleveland Clinic, an IHG Hotel', 'Price': 252, 'Rating': 'Scored 7.97.9Good625 reviews', 'Room Type': 'Ad'}
{'Name': 'Hampton Inn Cleveland-Downtown', 'Price': 253, 'Rating': 'Scored 8.08.0Very Good1,038 reviews', 'Room Type': 'Downtown Cleveland, ClevelandShow on map0.3 miles from downtown'}
{'Name': 'Hotel Indigo Cleveland Downtown, an IHG Hotel', 'Price': 297, 'Rating': 'Scored 8.18.1Very Good1,085 reviews', 'Room Type': 'Downtown Cleveland, ClevelandShow on map0.6 miles from downtown'}
{'Name': 'Tru By Hilton Cleveland Midtown', 'Price': 278, 'Rating': 'Scored 8.28.2Very Good1,217 reviews', 'Room Type': 'ClevelandShow on map2.7 miles from downtown'}
{'Name': 'Hilton Garden Inn Cleveland Downtown', 'Price': 318, 'Rating': 'Scored 6.96.9Review

In [11]:
schedule = pd.read_csv("24nflschedule.csv", delim_whitespace=True)
schedule = pd.DataFrame(schedule)
schedule
#for i in nfc_north_codes:
  #  print(schedule[schedule['TEAM'] == i])

/var/folders/br/_r58062114d6ttjjpchcm7z80000gn/T/ipykernel_9082/2087959862.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  schedule = pd.read_csv("24nflschedule.csv", delim_whitespace=True)


,TEAM,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,ARI,@BUF,LAR,DET,WSH,@SF,@GB,LAC,@MIA,CHI,NYJ,BYE,@SEA,@MIN,SEA,NE,@CAR,@LAR,SF
1,ATL,PIT,@PHI,KC,NO,TB,@CAR,SEA,@TB,DAL,@NO,@DEN,BYE,LAC,@MIN,@LV,NYG,@WSH,CAR
2,BAL,@KC,LV,@DAL,BUF,@CIN,WSH,@TB,@CLE,DEN,CIN,@PIT,@LAC,PHI,BYE,@NYG,PIT,@HOU,CLE
3,BUF,ARI,@MIA,JAX,@BAL,@HOU,@NYJ,TEN,@SEA,MIA,@IND,KC,BYE,SF,@LAR,@DET,NE,NYJ,@NE
4,CAR,@NO,LAC,@LV,CIN,@CHI,ATL,@WSH,@DEN,NO,NYG,BYE,KC,TB,@PHI,DAL,ARI,@TB,@ATL
5,CHI,TEN,@HOU,@IND,LAR,CAR,JAX,BYE,@WSH,@ARI,NE,GB,MIN,@DET,@SF,@MIN,DET,SEA,@GB
6,CIN,NE,@KC,WSH,@CAR,BAL,@NYG,@CLE,PHI,LV,@BAL,@LAC,BYE,PIT,@DAL,@TEN,CLE,DEN,@PIT
7,CLE,DAL,@JAX,NYG,@LV,@WSH,@PHI,CIN,BAL,LAC,BYE,@NO,PIT,@DEN,@PIT,KC,@CIN,MIA,@BAL
8,DAL,@CLE,NO,BAL,@NYG,@PIT,DET,BYE,@SF,@ATL,PHI,HOU,@WSH,NYG,CIN,@CAR,TB,@PHI,WSH
9,DEN,@SEA,PIT,@TB,@NYJ,LV,LAC,@NO,CAR,@BAL,@KC,ATL,@LV,CLE,BYE,IND,@LAC,@CIN,KC
